In [1]:
import numpy as np
import matplotlib.pyplot as plt

def NewtonianForce(mass1,mass2, r):
    return (6.67408*10**-11)*mass1*mass2/r/r;

In [2]:
def euler(h,t, x,f):
    return (f(t+h,x+h)-f(t,h)/2./h)

In [3]:
def RK4(h,t,x,y,z,f): #not a finite difference so no step in y
    k1= h*f(t,x,y,z)
    k2=h*f(t+h/2.,x,y,z)
    k3=h*f(t+h/2.,x,y,z)
    k4=h*f(t+h,x,y,z)
    return t+h, f(t,x,y,z)+1/6.*(k1+2.*k2+2.*k3+k4)



In [4]:
def polynomial4(t,x,a,b,c,d,e):
    p=[]
    for xi in x:
        pi=a*xi**4+b*xi**3+c*xi**2+d*xi+e
        #print(pi)
        p.append(pi)
    return np.array(p)
    

def polymaker(t,x,y,z,a,b,c,d,e):
    def poly(t,x,y,z):
        return polynomial4(t,x,a,b,c,d,e)
    return poly
        
        

In [5]:
import numpy as np
t=0.
x=0.01*np.arange(10.)
y=0.01*np.arange(10.)
z=0.01*np.arange(10.)
print(t,x)
polyfn=polymaker(t,x,y,z,1.,0.,0.,0.,0.)

0.0 [0.   0.01 0.02 0.03 0.04 0.05 0.06 0.07 0.08 0.09]


In [6]:
#import matplotlib.pyplot as plt
#plt.plot(t,polyfn(t,x))

In [7]:
outp=polyfn(t,x,y,z) #this is correct to order of magnitude
print(outp)

[0.000e+00 1.000e-08 1.600e-07 8.100e-07 2.560e-06 6.250e-06 1.296e-05
 2.401e-05 4.096e-05 6.561e-05]


In [8]:
from bokeh.plotting import figure, output_notebook, show
# create a new plot with a title and axis labels

output_notebook()
p = figure(title="Polynomial function (x^4)", x_axis_label='x', y_axis_label='y')

# add a line renderer with legend and line thickness
p.line(x, y, legend="Quartic polynomial", line_width=2)

# show the results
show(p)

Loading BokehJS ...

In [9]:
rk4polyout=RK4(.01,t,x,y,z,polyfn)
print(outp+x)

[0.         0.01000001 0.02000016 0.03000081 0.04000256 0.05000625
 0.06001296 0.07002401 0.08004096 0.09006561]


In [10]:
print(rk4polyout) 
#There are multiple items in the y output, which is confusing. trace this over time and plot an evolution and compare to exact solution, which I have analytically written down. steve points out truncation error
print(((rk4polyout[1]-x-outp)/x))
#order4 polynomial error (comparable in size to polyfn itself, relative error with it is one)

(0.01, array([0.00000e+00, 1.01000e-08, 1.61600e-07, 8.18100e-07, 2.58560e-06,
       6.31250e-06, 1.30896e-05, 2.42501e-05, 4.13696e-05, 6.62661e-05]))
[        nan -0.99999999 -0.99999992 -0.99999973 -0.99999936 -0.99999875
 -0.99999784 -0.99999657 -0.99999488 -0.99999271]


/Users/sdorsher/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
def invpolyint(t,x):
    p=[]
    for ti in t:
        xi=-3*t**(-1./3.)
        p.append(xi)
    return np.array(p)
    



In [12]:
#print(invpolyint(t,x)) #need an entire run to invert this because you need a history of a timeseries accumulated

In [13]:
def analyticalpolyint(t,x):
    p=[]
    for xi in x:
        ti = -1./3.*xi**(-3.)
        p.append(ti)
    return np.array(p)

In [14]:
print(x)

[0.   0.01 0.02 0.03 0.04 0.05 0.06 0.07 0.08 0.09]


In [15]:
#print(analyticalpolyint(t,x))

In [16]:
def invertIMF(luminosity,lsun,masssun):
    M=[]
    for L in luminosity: 
        if (L<(.43)**4)*lsun:
            x=((L/lsun/0.23)**(1./2.3))*masssun
            #print(x,1)
            M.append(x)
        elif (L<16*lsun):
            x=(((L/lsun)**(0.25))*masssun)
            #print(L,lsun,masssun,x,2)
            M.append(x)
        elif (L<(((20**(3.5))*1.4)*masssun)):
            x=(((L/lsun/1.4)**(1./3.5))*masssun)
            #print(x,3)
            M.append(x)
        else:
            x=((L/lsun/32000.)*masssun)
            #print(x,4)
            M.append(x)
    return np.array(M)
    

In [17]:
import math
def InitialData():
    random.seed(a=9001)
    
    #initially use an in plane orbit with random starting locations relative to the x axis
    phi=np.array([0,math.pi])
    orbitangle=np.zeros(2)
    #orbitalradius=np.random.uniform(.1,50,2)
    orbitalradius=np.ones(2)
    #start with circular orbits
    #eccentricity=np.random.uniform(0.,.1,2)
    eccentricity=np.zeros(2)
    #magnitude=np.random.uniform(-20,-30,2) #absolute not apparent maginutde
    #magsun=-26.832
    masssun=1.989*10**30
    lsun=3.828*10**26
    massearth=5.9722*10**24
    #luminosity=lsun*10**(0.4*(magnitude-magsun))
    #masses= invertIMF(luminosity,lsun,masssun) #Initial mass function for Main Sequence
    #masses=np.random.uniform(.7,5.) #replace with IMF
    masses=np.array([masssun,massearth])

    return phi,orbitangle,orbitalradius,eccentricity, masses



In [18]:
import random
initdat=InitialData()

In [19]:
print(initdat)

(array([0.        , 3.14159265]), array([0., 0.]), array([1., 1.]), array([0., 0.]), array([1.9890e+30, 5.9722e+24]))


In [20]:
def getxyuv(initdat):
    phi,orbitangle,orbitalradius,eccentricity, masses=initdat
    #print(orbitalradius, phi, np.cos(phi), np.sin(phi))
    metersperAU=149597870700
    Gconstant=6.408*10**-11
    x0=orbitalradius*np.cos(phi)*metersperAU
    y0=orbitalradius*np.sin(phi)*metersperAU
    z0=np.zeros(2)
    

    
    #start at perihelion for both (eliptical, doesn't generalize to three body)
    #actually start with circular orbit
    ux0=np.zeros(2)*149597870700
    #centrepital force balances gravitational force
    reducedmass=np.zeros(2)
    print(masses)
    for i in np.arange(2):
        j=(i+1)%2 #reverse masses
        reducedmass[i]=masses[i]*masses[j]/np.sum(masses)
    print(reducedmass)
    rphys=orbitalradius*metersperAU*reducedmass/np.sum(masses)
    print(rphys)
    F=(Gconstant*reducedmass**2/rphys**2)
    print(F)
    #centF=reducedmass*v**2/rphys
    #centF=accel
    v=np.sqrt(Gconstant*reducedmass/rphys)
    print(v)
    ux0=v*np.sin(phi)
    uy0=v*np.cos(phi) #initial data in y only
    #evolve in plane only
    #there is a units problem that needs to be fixed
    #velocity initial conditions are not trivial. 
    uz0=np.zeros(2)
    
    #circular orbit   #a=omega^2 * r #v=omega*r #omega=v/r
    omega=v/rphys
    print(omega)
    omegatrue=np.mean(omega) #shouldaccount for numerical effects
    print(omegatrue)
    a=omegatrue**2*rphys
    print("a",a)
    print(phi)
    ax0=-a*np.cos(phi)
    ay0=-a*np.sin(phi)
    az0=np.zeros(2)
    
    
    return reducedmass,x0,y0,z0, ux0, uy0,uz0, ax0, ay0,az0

In [21]:
xyuva=getxyuv(initdat)
print(xyuva)#In SI units
print(xyuva[1][0])
print(xyuva[0][0]/xyuva[0][1])
print(xyuva)

[1.9890e+30 5.9722e+24]
[5.97218207e+24 5.97218207e+24]
[449182.02019948 449182.02019948]
[1.13277552e+28 1.13277552e+28]
[29188.82340899 29188.82340899]
[0.06498217 0.06498217]
0.06498217225174459
a [1896.75315059 1896.75315059]
[0.         3.14159265]
(array([5.97218207e+24, 5.97218207e+24]), array([ 1.49597871e+11, -1.49597871e+11]), array([0.00000000e+00, 1.83204554e-05]), array([0., 0.]), array([0.00000000e+00, 3.57459992e-12]), array([ 29188.82340899, -29188.82340899]), array([0., 0.]), array([-1896.75315059,  1896.75315059]), array([-0.00000000e+00, -2.32285267e-13]), array([0., 0.]))
149597870700.0
1.0
(array([5.97218207e+24, 5.97218207e+24]), array([ 1.49597871e+11, -1.49597871e+11]), array([0.00000000e+00, 1.83204554e-05]), array([0., 0.]), array([0.00000000e+00, 3.57459992e-12]), array([ 29188.82340899, -29188.82340899]), array([0., 0.]), array([-1896.75315059,  1896.75315059]), array([-0.00000000e+00, -2.32285267e-13]), array([0., 0.]))


In [22]:
def timestep(step,t,dt,reducedmass,xi,yi,zi, vxi, vyi, vzi, axi, ayi, azi):
    xii=np.zeros(np.size(xi))
    vxii=np.zeros(np.size(vx))
    yii=np.zeros(np.size(yi))
    vyii=np.zeros(np.size(vy))
    zii=np.zeros(np.size(vzi))
    vzii=np.zeros(np.size(vzi))
    rii=np.zeros(np.size(xi))
    axii=axi
    ayii=ayi
    azii=azi
    
    for m in np.arange(len(x)):
        #m represents choices of mass
        i=step
        
        xii[m] = xi[m] + dt*vxi[m]
        #print(xii)
        vxii[m] = vxi[m] + dt*axi[m]
        #print(vxii)
        yii[m]= yi[m] + dt*vyi[m]
        vyii[m] = vyi[m] + dt*ayi[m]
        zii[m]= zi[m] + dt*vzi[m]
        vzii[m] = vzi[m] + dt*azi[m]
        rii[m]=np.sqrt(xi[m]**2+yi[m]**2+zi[m]**2)
    
    
    Gconstant=6.408*10**-11
    for k in np.arange(len(rii)):
        for j in np.arange(len(rii)):
            if j!=k:
                rreljk=np.abs((xi[j] - xi[k])**2+(yi[j]-yi[k])**2+(zi[j]-zi[k])**2)**(1./2.)
                axii[j]+=Gconstant*reducedmass[k]*(xi[j]  - xi[k])/rreljk**3
                ayii[j]+=Gconstant*reducedmass[k]*(yi[j]  - yi[k])/rreljk**3
                azii[j]+=Gconstant*reducedmass[k]*(zi[j]  - zi[k])/rreljk**3
    #print(xii)
    return reducedmass, xii,yii,zii,vxii,vyii,vzii,axii,ayii,azii
                    

In [23]:
dt=0.01*31556926 #seconds per year
numsteps=5
mass,x,y,z,vx,vy,vz,ax,ay,az=xyuva
for i in np.arange(1,numsteps):
    t=0.+i*numsteps*dt
    mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(i,t,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
    print(x,y,vx,vy,ax,ay)
    #print(ay) #forces should be equal and opposite, but in reduced mass framework accelerations are also equal and opposite
    #accelerations should evolve from y to x with time in a sinusoidal manner even in reduced mass framework
    #print(ax)
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(2,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(3,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(4,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 

[ 1.49597871e+11 -1.49597871e+11] [ 9.2110954e+09 -9.2110954e+09] [-5.98556988e+08  5.98556988e+08] [ 29188.82340899 -29188.82340906] [-1896.75315058  1896.75315058] [-2.61773442e-25 -2.32285267e-13]
[-1.88736588e+14  1.88736588e+14] [ 1.84221908e+10 -1.84221908e+10] [-1.19711398e+09  1.19711398e+09] [ 29188.82340899 -29188.82340914] [-1896.75315058  1896.75315058] [ 2.61737257e-10 -2.61969543e-10]
[-5.6650896e+14  5.6650896e+14] [ 2.76332862e+10 -2.76332862e+10] [-1.79567096e+09  1.79567096e+09] [ 29188.82349159 -29188.82349181] [-1896.75315058  1896.75315058] [ 2.61737258e-10 -2.61969543e-10]
[-1.13316752e+15  1.13316752e+15] [ 3.68443816e+10 -3.68443816e+10] [-2.39422795e+09  2.39422795e+09] [ 29188.82357418 -29188.82357447] [-1896.75315058  1896.75315058] [ 2.61737258e-10 -2.61969543e-10]


In [24]:
class OrbitDiffEq:
    def __init__(self,reducedmass,x0,y0,z0, ux0, uy0,uz0, ax0, ay0,az0):
        self.reducedmass=reducedmass
        self.xi=x0
        self.yi=y0
        self.zi=z0
        self.vxi=ux0
        self.vyi=uy0
        self.vzi=uz0
        self.axi=ax0
        self.ayi=ay0
        self.azi=az0
    def dxidt(self,t):
        return self.vxi
    def dyidt(self,t):
        return self.vyi
    def dzidt(self,t):
        return self.vzi
    def dvxidt(self,t):
        #return axi[m]
        axii=self.axi
        rii=np.sqrt(self.xi**2+self.yi**2+self.zi**2)
        Gconstant=6.408*10**-11
        for k in np.arange(len(rii)):
            for j in np.arange(len(rii)):
                if j!=k:
                    rreljk=np.abs((self.xi[j] - self.xi[k])**2+(self.yi[j]-self.yi[k])**2+(self.zi[j]-self.zi[k])**2)**(1./2.)
                    axii[j]+=Gconstant*self.reducedmass[k]*(self.xi[j]  - self.xi[k])/rreljk**3
        self.axi=axii
        return axii
    def dvyidt(self,t):
        #return axi[m]
        ayii=self.ayi
        rii=np.sqrt(self.xi**2+self.yi**2+self.zi**2)
        Gconstant=6.408*10**-11
        for k in np.arange(len(rii)):
            for j in np.arange(len(rii)):
                if j!=k:
                    rreljk=np.abs((self.xi[j] - self.xi[k])**2+(self.yi[j]-self.yi[k])**2+(self.zi[j]-self.zi[k])**2)**(1./2.)
                    ayii[j]+=Gconstant*self.reducedmass[k]*(self.xi[j]  - self.xi[k])/rreljk**3
        self.ayi=ayii
        return ayii
    def dvzidt(self,t):
        #return axi[m]
        azii=self.azi
        rii=np.sqrt(self.xi**2+self.yi**2+self.zi**2)
        Gconstant=6.408*10**-11
        for k in np.arange(len(rii)):
            for j in np.arange(len(rii)):
                if j!=k:
                    rreljk=np.abs((self.xi[j] - self.xi[k])**2+(self.yi[j]-self.yi[k])**2+(self.zi[j]-self.zi[k])**2)**(1./2.)
                    azii[j]+=Gconstant*self.reducedmass[k]*(self.xi[j]  - self.xi[k])/rreljk**3
        self.azi=azii
        return azii
    def updateINTERNAL(self,reducedmass, xii,yii,zii,vxii,vyii,vzii):
        self.xi=xii
        self.yi=yii
        self.zi=zii
        self.vxi=vxii
        self.vyi=vyii
        self.vzi=vzii
        return self
    def update(self,reducedmass,xii,yii,zii,vxii,vyii,vzii,axii,ayii,azii):
        self.xi=xii
        self.yi=yii
        self.zi=zii
        self.vxi=vxii
        self.vyi=vyii
        self.vzi=vzii
        self.axi=axii
        self.ayi=ayii
        self.azi=azii
    def print2D(self):
        print(self.xi,self.yi,self.vxi,self.vyi,self.axi,self.ayi)
        return self
    def list2D(self):
        return self.xi,self.yi,self.vxi,self.vyi,self.axi,self.ayi
    def timestepRK4ODE(self,step,t,dt):

    
        h=dt
        #tnew,ynew, intval=RK4(h,t,y,f)
        #m represents choices of mass
        i=step
        
        tnew,intvalx=RK4implicit(h,t,self.dxidt)
        xii = intvalx
        tnew,intvalvx=RK4implicit(h,t,self.dvxidt)
        vxii=intvalvx
        tnew,intvaly=RK4implicit(h,t,self.dyidt)
        yii = intvaly
        tnew,intvalvy=RK4implicit(h,t,self.dvyidt)
        vyii=intvalvy
        tnew,intvalz=RK4implicit(h,t,self.dzidt)
        zii = intvalz
        tnew,intvalvz=RK4implicit(h,t,self.dvzidt)
        vzii=intvalvz
 
        #print(xii)
        self.updateINTERNAL(reducedmass, xii,yii,zii,vxii,vyii,vzii)
        return reducedmass, xii,yii,zii,vxii,vyii,vzii,self.axi,self.ayi,self.azi

In [25]:
def RK4implicit(h,t,f): #not a finite difference so no step in y
    k1= h*f(t)
    k2=h*f(t+h/2.)
    k3=h*f(t+h/2.)
    k4=h*f(t+h)
    return t+h, f(t)+1/6.*(k1+2.*k2+2.*k3+k4)




In [26]:
reducedmass,x0,y0,z0, ux0, uy0,uz0, ax0, ay0,az0=xyuva
ODE= OrbitDiffEq(reducedmass,x0,y0,z0, ux0, uy0,uz0, ax0, ay0,az0)

In [27]:
mass,x,y,z,vx,vy,vz,ax,ay,az=xyuva
print(x,y,vx,vy,ax,ay)
print(xyuva)

[ 1.49597871e+11 -1.49597871e+11] [0.00000000e+00 1.83204554e-05] [0.00000000e+00 3.57459992e-12] [ 29188.82340899 -29188.82340899] [-1896.75315058  1896.75315058] [ 2.61737258e-10 -2.61969543e-10]
(array([5.97218207e+24, 5.97218207e+24]), array([ 1.49597871e+11, -1.49597871e+11]), array([0.00000000e+00, 1.83204554e-05]), array([0., 0.]), array([0.00000000e+00, 3.57459992e-12]), array([ 29188.82340899, -29188.82340899]), array([0., 0.]), array([-1896.75315058,  1896.75315058]), array([ 2.61737258e-10, -2.61969543e-10]), array([0., 0.]))


In [28]:
dt=0.01*31556926 #seconds per year
numsteps=5
reducedmass,x,y,z,vx,vy,vz,ax,ay,az=xyuva
for i in np.arange(1,numsteps):
    t=0.+i*numsteps*dt
    reducedmass, x,y,z,vx,vy,vz,ax,ay,az=ODE.timestepRK4ODE(i,t,dt)
    ODE.print2D()
    ODE.update(reducedmass, x,y,z,vx,vy,vz,ax,ay,az)



[0.00000000e+00 1.12803742e-06] [ 9.21112459e+09 -9.21112459e+09] [-5.98558885e+08  5.98558885e+08] [ 0.00345533 -0.0034554 ] [-1896.75315056  1896.75315056] [ 2.16371609e-08 -2.16373932e-08]
[-1.88887383e+14  1.88887383e+14] [ 1090.39975975 -1090.42289179] [-5.98558885e+08  5.98558885e+08] [ 0.00682804 -0.00682812] [-1896.75315056  1896.75315056] [ 2.16371609e-08 -2.16373932e-08]
[-1.88887383e+14  1.88887383e+14] [ 2154.72777683 -2154.75090886] [-5.98558885e+08  5.98558885e+08] [ 0.00682804 -0.00682812] [-1896.75315056  1896.75315056] [ 2.16371475e-08 -2.16373798e-08]
[-1.88887383e+14  1.88887383e+14] [ 2154.72710922 -2154.75024125] [-5.98558885e+08  5.98558885e+08] [ 0.00682804 -0.00682811] [-1896.75315056  1896.75315056] [ 2.16371341e-08 -2.16373664e-08]


In [29]:
secperyr=31556926

In [30]:
dt=0.01*31556926 #seconds per year
numsteps=5
reducedmass,x,y,z,vx,vy,vz,ax,ay,az=xyuva
datainit=[[0,0.0,reducedmass,x,y,z,vx,vy,vz,ax,ay,az]]
import pandas as pd
columns =['step','time','reducedmass','x' ,'y','z','vx','vy','vz','ax','ay','az']
df = pd.DataFrame(data=datainit, columns=columns)




In [31]:
display(df)

,step,time,reducedmass,x,y,z,vx,vy,vz,ax,ay,az
0,0,0.0,"[5.972182067840249e+24, 5.972182067840249e+24]","[149597870700.0, -149597870700.0]","[0.0, 1.832045535120146e-05]","[0.0, 0.0]","[0.0, 3.574599915869588e-12]","[29188.823408989138, -29188.823408989138]","[0.0, 0.0]","[-1896.7531505587856, 1896.7531505587856]","[2.1637134099864023e-08, -2.1637366385131486e-08]","[2.1375396842196047e-08, -2.1375396842196047e-08]"


In [32]:
for i in np.arange(1,numsteps):
    t=0.+i*numsteps*dt
    reducedmass, x,y,z,vx,vy,vz,ax,ay,az=ODE.timestepRK4ODE(i,t,dt)
    df.loc[i]=i,t,reducedmass,x,y,z,vx,vy,vz,ax,ay,az
    ODE.update(reducedmass, x,y,z,vx,vy,vz,ax,ay,az)


In [33]:
display(df)

,step,time,reducedmass,x,y,z,vx,vy,vz,ax,ay,az
0,0,0.0,"[5.972182067840249e+24, 5.972182067840249e+24]","[149597870700.0, -149597870700.0]","[0.0, 1.832045535120146e-05]","[0.0, 0.0]","[0.0, 3.574599915869588e-12]","[29188.823408989138, -29188.823408989138]","[0.0, 0.0]","[-1896.7531505587856, 1896.7531505587856]","[2.1637080468400597e-08, -2.163731275366806e-08]","[2.137534321073262e-08, -2.137534321073262e-08]"
1,1,1577846.3,"[5.972182067840249e+24, 5.972182067840249e+24]","[-188887382926151.7, 188887382926151.7]","[2154.725774002702, -2154.748906035592]","[2128.660776772775, -2128.660776772775]","[-598558884.8776551, 598558884.8776551]","[0.006828033917980395, -0.006828107220302642]","[0.006745437423526423, -0.006745437423526423]","[-1896.7531505587856, 1896.7531505587856]","[2.1637080468400597e-08, -2.163731275366806e-08]","[2.137534321073262e-08, -2.137534321073262e-08]"
2,2,3155692.6,"[5.972182067840249e+24, 5.972182067840249e+24]","[-188887382926151.7, 188887382926151.7]","[2154.724438785892, -2154.747570818782]","[2128.6594415559634, -2128.6594415559634]","[-598558884.8776551, 598558884.8776551]","[0.00682802968685668, -0.006828102989178927]","[0.00674543319240271, -0.00674543319240271]","[-1896.7531505587856, 1896.7531505587856]","[2.1637080468400597e-08, -2.163731275366806e-08]","[2.137534321073262e-08, -2.137534321073262e-08]"
3,3,4733538.9,"[5.972182067840249e+24, 5.972182067840249e+24]","[-188887382926151.7, 188887382926151.7]","[2154.7231035690806, -2154.746235601971]","[2128.6581063391536, -2128.6581063391536]","[-598558884.8776551, 598558884.8776551]","[0.006828025455732966, -0.006828098758055213]","[0.006745428961278994, -0.006745428961278994]","[-1896.7531505587856, 1896.7531505587856]","[2.1637080468400597e-08, -2.163731275366806e-08]","[2.137534321073262e-08, -2.137534321073262e-08]"
4,4,6311385.2,"[5.972182067840249e+24, 5.972182067840249e+24]","[-188887382926151.7, 188887382926151.7]","[2154.721768352271, -2154.7449003851607]","[2128.6567711223424, -2128.6567711223424]","[-598558884.8776551, 598558884.8776551]","[0.006828021224609252, -0.006828094526931499]","[0.006745424730155281, -0.006745424730155281]","[-1896.7531505587856, 1896.7531505587856]","[2.1637080468400597e-08, -2.163731275366806e-08]","[2.137534321073262e-08, -2.137534321073262e-08]"


In [34]:
xser=df['x']
xser
xser[1][0]

-188887382926151.7

In [35]:
def mergeStarDataIntoDataFrame(col):
    rawdataser=df[col]
    numstars=len(rawdataser[0])
    columns=np.arange(numstars)
    numentries=len(rawdataser)
    index=np.arange(numentries)
    dfnew=pd.DataFrame(columns=columns,index=index)
    for coln in np.arange(numstars):
        datacollist=[]
        for row in np.arange(numentries):
            datacollist.append(rawdataser[row][coln])
        dfnew[coln]=datacollist
    return dfnew

In [36]:
dfx=mergeStarDataIntoDataFrame('x')

In [37]:
display(dfx)

,0,1
0,1.495979e+11,-1.495979e+11
1,-1.888874e+14,1.888874e+14
2,-1.888874e+14,1.888874e+14
3,-1.888874e+14,1.888874e+14
4,-1.888874e+14,1.888874e+14


In [38]:
dfy=mergeStarDataIntoDataFrame('y')

In [39]:
display(dfy)

,0,1
0,0.000000,0.000018
1,2154.725774,-2154.748906
2,2154.724439,-2154.747571
3,2154.723104,-2154.746236
4,2154.721768,-2154.744900


In [40]:
dfvx=mergeStarDataIntoDataFrame('vx')

In [41]:
display(dfvx)

,0,1
0,0.000000e+00,3.574600e-12
1,-5.985589e+08,5.985589e+08
2,-5.985589e+08,5.985589e+08
3,-5.985589e+08,5.985589e+08
4,-5.985589e+08,5.985589e+08


In [42]:
dfvy=mergeStarDataIntoDataFrame('vy')

In [43]:
display(dfvy)

,0,1
0,29188.823409,-29188.823409
1,0.006828,-0.006828
2,0.006828,-0.006828
3,0.006828,-0.006828
4,0.006828,-0.006828


In [44]:
dfax=mergeStarDataIntoDataFrame('ax')

In [45]:
display(dfax)

,0,1
0,-1896.753151,1896.753151
1,-1896.753151,1896.753151
2,-1896.753151,1896.753151
3,-1896.753151,1896.753151
4,-1896.753151,1896.753151


In [46]:
dfay=mergeStarDataIntoDataFrame('ay')

In [47]:
display(dfay)

,0,1
0,2.163708e-08,-2.163731e-08
1,2.163708e-08,-2.163731e-08
2,2.163708e-08,-2.163731e-08
3,2.163708e-08,-2.163731e-08
4,2.163708e-08,-2.163731e-08


In [51]:
df['time'].tolist()

[0.0, 1577846.3, 3155692.6, 4733538.9, 6311385.2]